# Setup Enviorment

In [1]:
from Preprocessing.selfsupervised_data_praparation import plot_samples, get_dataset_list
from Preprocessing.generate_embeddings import generate_embeddings_df, save_embeddings_as_csv
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50V2
from tensorflow.keras.applications.convnext import ConvNeXtBase, ConvNeXtSmall, ConvNeXtTiny

import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 

2023-06-12 19:54:35.643547: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-12 19:54:35.681562: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-12 19:54:35.682811: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 19:54:36.480462: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Path to Dataset
path = '../Dataset_10_best_cities/'
ignore_black = False

model_name = 'autoencoder' #'VGG16' #'MobileNetV2' # 'vit' # 'autoencoder' #'variational_autoencoder' # 'ResNet50V2' #'ConvNeXtTiny'
encoder_backbone = 'ConvNeXtTiny' # 'vit' # 'ResNet50V2' # 'ConvNeXtTiny'
latent_dim = 1024

target_size = (224, 224, 3)
model_input = (224, 224, 3)
band = None

# Model path _full_dataset
model_path = f'Weights/{model_name}_{encoder_backbone}_{target_size[0]}_{latent_dim}_{target_size[2]}Bands_full_dataset.h5'  

if target_size[2] == 1:
    # Embeddings path
    if model_name in ['autoencoder', 'variational_autoencoder']:
        embeddings_path = f'Embeddings/{model_name}/{model_name}_{encoder_backbone}_Per_Band/{target_size[0]}_band{band+1}.csv'
    else:
        embeddings_path = f'Embeddings/{model_name}_Per_Band/{target_size[0]}_band{band+1}.csv'
else:
    # Embeddings path
    if model_name in ['autoencoder', 'variational_autoencoder']:
        embeddings_path = f'Embeddings/{model_name}/{model_name}_{encoder_backbone}__{target_size[0]}_{latent_dim}_{target_size[2]}Bands.csv'
    else:
        embeddings_path = f'Embeddings/{model_name}_{target_size[0]}_{latent_dim}.csv'

if ignore_black:
    embeddings_path = embeddings_path.replace(".csv", "_no_black_images.csv")    
    

## Read dataset

In [3]:
#plot_samples(path)

In [4]:
image_list = get_dataset_list(path, ignore_black=ignore_black, show_dirs=True, head=0)

Images in directories: 
54001
41001
5001
50001
68001
8001
23001
76001
73001
5360


## Load Model

In [5]:
tf.random.set_seed(0)

if model_name == 'variational_autoencoder':
    from Models import Variational_Autoencoder
    model = Variational_Autoencoder.get_Variational_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim, lr=0.0001, encoder_backbone=encoder_backbone)
elif model_name == 'autoencoder':
    from Models import Autoencoder
    model = Autoencoder.get_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim, encoder_backbone=encoder_backbone)
elif model_name == 'vit':
    from Models import ViT
    model = ViT.get_vit_backbone(model_input)
elif model_name == 'MobileNetV2':
    cnn = MobileNetV2(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'VGG16': # min depth
    cnn = VGG16(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'ResNet50V2':
    cnn = ResNet50V2(input_shape=model_input, include_top=False, weights='imagenet') 
elif model_name == 'ConvNeXtTiny':
    cnn = ConvNeXtTiny(input_shape=model_input, include_top=False, weights='imagenet')  

if model_name in ['MobileNetV2', 'VGG16', 'ResNet50V2', 'ConvNeXtTiny']:
    model = Sequential()
    model.add(cnn)
    model.add(tf.keras.layers.GlobalAveragePooling2D())

    # freeze:
    for layer in model.layers:
        layer.trainable = False
        
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 1024)              28607584  
                                                                 
Total params: 28,607,584
Trainable params: 0
Non-trainable params: 28,607,584
_________________________________________________________________


In [7]:
#from huggingface_hub import push_to_hub_keras

#from huggingface_hub import push_to_hub_keras
#push_to_hub_keras(model, 'MITCriticalData/Sentinel-2_ConvNeXtTiny_Autoencoder_RGB_full_Colombia_Dataset', create_pr=1)

## Generate embeddings

In [ ]:
if target_size[2] == 1:
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size, BAND=band)
else:
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size)
embeddings

In [ ]:
save_embeddings_as_csv(df=embeddings, path=embeddings_path)

In [ ]:
pd.read_csv(embeddings_path)